# Post-processing of case studies

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

I run this notebook on Ubuntu, using this version of matplotlib:
```
pip freeze | grep matplotlib
matplotlib==2.1.0
```
Some older versions return an error because they do not support some plot configurations.
I set in the virtual machine the RAM to 12GB as 8GB is not sufficient to parse two annual result files.

## Import required libraries

In [50]:
import os
import cases

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

## Load in simulation results

In [51]:
# Helper methods

def get_results(case_name):
    """ Get the results for the case with name `case_name`
    """
    # Make sure simulation was successful
    dslog_name = os.path.join("simulations", case_name, "dslog.txt")
    with open(dslog_name) as dslog:
        if not "Integration terminated successfully" in dslog.read():
            raise Exception("Simulation failed. Check {}".format(dslog_name))
    file_name = cases.get_result_file_name(case_name)
    return Reader(file_name, "dymola")

def get_partial_results(case_name, list_of_variables):
    """ Get a dictionary with the variable names and the time series for `list_of_variables`
    """
    reader = get_results(case_name)
    d = dict()
    read_time = True
    for v in list_of_variables:
        if read_time:
            d['time'] = reader.values(v)[0]
            read_time = False
        d[v] = reader.values(v)[1]
    return d

In [52]:
all_case_names = cases.get_list_of_case_names()

res = dict()

for case_name in all_case_names:
    res[case_name] = get_results(case_name)

print("Read in simulation results.")

Read in simulation results.


In [63]:
# Look at available resulting variables
res['test_base'].varNames()

(array([0.0000000e+00, 1.7588939e+02, 1.7588939e+02, 6.2398889e+02,
        6.2398889e+02, 1.2000000e+03, 1.2000000e+03, 1.2008291e+03,
        1.2008291e+03, 1.7280000e+03, 3.4560000e+03, 5.1840000e+03,
        6.9120000e+03, 8.6400000e+03, 1.0368000e+04, 1.2096000e+04,
        1.3824000e+04, 1.5552000e+04, 1.7280000e+04, 1.9008000e+04,
        2.0736000e+04, 2.2464000e+04, 2.4192000e+04, 2.5920000e+04,
        2.7648000e+04, 2.9376000e+04, 3.1104000e+04, 3.2832000e+04,
        3.4560000e+04, 3.6000000e+04, 3.6000000e+04, 3.6288000e+04,
        3.8016000e+04, 3.9744000e+04, 4.1472000e+04, 4.3200000e+04,
        4.4928000e+04, 4.6656000e+04, 4.8384000e+04, 5.0112000e+04,
        5.1840000e+04, 5.3568000e+04, 5.5296000e+04, 5.7024000e+04,
        5.8752000e+04, 6.0480000e+04, 6.2208000e+04, 6.3936000e+04,
        6.5664000e+04, 6.7392000e+04, 6.9120000e+04, 7.0848000e+04,
        7.2576000e+04, 7.4304000e+04, 7.6032000e+04, 7.7760000e+04,
        7.9488000e+04, 8.1216000e+04, 8.2944000e

## Configure plots

In [53]:
plt.rcParams['axes.facecolor']='whitesmoke'
plt.rcParams['font.size'] = 6
plt.rcParams['text.usetex'] = False
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.framealpha'] = 0.75
plt.rcParams['legend.edgecolor'] = 'none'
plt.rcParams['savefig.dpi'] = 300

def save_plot(figure, file_name):
    """ Save the figure to a pdf and png file in the directory `img`
    """
    import os
    import matplotlib.pyplot as plt
    
    out_dir = "img"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    figure.savefig(os.path.join(out_dir, '{}.pdf'.format(file_name)))
    figure.savefig(os.path.join(out_dir, '{}.png'.format(file_name)))
    plt.clf()
    

def configure_axes(axes):
    """ Configure the axis style
    """
    axes.spines['right'].set_visible(False)
    axes.spines['top'].set_visible(False)
    axes.spines['left'].set_visible(False)
    axes.spines['bottom'].set_visible(False)
    axes.grid(color='lightgrey', linewidth=0.25)
    return


# ---------------------------------------------------------------------------
# helper functions and scripts

def set_cases_and_initiate_plot():
    from matplotlib.gridspec import GridSpec
    cases = ['test_base', 'test_1711']
    seasons = ['']
    num_cases = len(cases)
    num_seasons = len(seasons)
    
    fig = plt.figure(figsize=(6.5,8.))
    gs1 = GridSpec(80, 1)
    gs1.update(left=0.1, right=0.9, hspace=0.05)
    
    ax = list()
    ax.insert(0, fig.add_subplot(gs1[0:11,:]))
    ax.insert(1, fig.add_subplot(gs1[12:23,:]))
    ax.insert(2, fig.add_subplot(gs1[28:39,:]))
    ax.insert(3, fig.add_subplot(gs1[40:51,:]))
    ax.insert(4, fig.add_subplot(gs1[56:67,:]))
    ax.insert(5, fig.add_subplot(gs1[68:79,:]))
    
    # fig, ax = plt.subplots(nrows=num_cases*num_seasons, ncols=1, figsize = (6.5,8.))
    # fig, ax = plt.subplots(nrows=20, ncols=1, figsize = (6.5,8.))
    
    return cases, seasons, num_cases, num_seasons, fig, ax

def set_title(ax, title):
    left, width = .01, .97
    bottom, height = .01, .88
    right = left + width
    top = bottom + height
    
    title_str = r"$\it{" + title + "}$"
    ax.text(left, top,
            title_str,
            verticalalignment = 'center',
            horizontalalignment = 'left', 
            transform=ax.transAxes,
            fontsize = 6, color = 'k',
            bbox=dict(facecolor='white', alpha=0.75, edgecolor='none'))
    
    
def set_up_labels(i, ax, cases, seasons, num_cases, num_seasons, x_axis_label, y_axis_label):
    # Hide xtick labels and ticks on the upper case subplot (each basecase)
    if i % 2 == 0:
        hide_tick_labels(ax)

    # Print x axis title only below the lowest subplot
    if i  == num_cases*num_seasons - 1:
        ax.set_xlabel(x_axis_label)
    ax.set_ylabel(y_axis_label)
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))
     
    # Annotate case
    set_title(ax, cases[i % 2])
    # Annotate case
    # if i % 2 == 0:
    #     title_str = r"$\bf{" + seasons[i/2] + "}$" + ' (upper: ' + r"$\it{" + cases[i % 2] + "}$" + ', lower: ' + r"$\it{"  + cases[(i-1) % 2] + "}$" + ')'
    #     ax.set_title(title_str, # mg assign appropriate season/case
    #                  verticalalignment = 'top',
    #                  horizontalalignment = 'center', 
    #                  fontsize = 6, color = 'k')
        
    # Print legend only at the lower plot (g36 case)
    if i % 1 == 0:
        ax.legend(loc='center right', ncol=1)
    configure_axes(ax)
        
    #plt.tight_layout(h_pad=0)
    plt.tight_layout()
    #plt.subplots_adjust(hspace = .2)
        
def tem_conv_CtoF(T_in_degC):
    '''Converts temperature provided in degC to degF
    '''
    T_in_degF = (T_in_degC)*9./5. + 32.
    
    return T_in_degF
        
def add_secondary_yaxis_for_degF(ax, time, temp_in_K):
        # Add a secondary axis with temperatures represented in F
        ax_F = ax.twinx()
        # Get limits to match with the left axis
        ax_F.set_ylim([tem_conv_CtoF(ax.get_ylim()[0]),tem_conv_CtoF(ax.get_ylim()[1])])
        # plot a "scaler" variable and make it invisible
        ax_F.plot(time, tem_conv_CtoF(temp_in_K-273.15), linewidth=0.0)
        ax_F.set_ylabel('temperature [$^\circ$F]')
        configure_axes(ax_F)
        #ax.grid(False)
        #ax.xaxis.grid()
        
def hide_tick_labels(ax):
    '''Removes labels and ticks. Kwargs: bottom controls the ticks, labelbottom the tick labels
    '''
    ax.tick_params(axis = 'x',labelbottom='off',bottom='off')

## Plot results

## Chiller plant - power

In [59]:
def plot_power(reader):
    ''' Main method that plots the results
    '''
    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 6}
    matplotlib.rc('font', **font)
    
    plt.clf()
    
    time_scale=3600.
       
    (t, pumCHW_P) = reader.values("pumCHW.P")
    (t, pumCW_P) = reader.values("pumCW.P")
    (t, chi_P) = reader.values("chi.P")
    t = t/time_scale
   
    # Plot figure
    fig = plt.figure(figsize=(6.5, 2.5))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(t, pumCHW_P, label = '$P_{chw,p}$', linewidth=0.5)
         
    #make_ticklabels_invisible(plt.gcf())
        
    # customize days to display

    ax.set_xlabel('time [h]')
    ax.set_ylabel('Chiller water pump power [W]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

    ax.set_xlim([min(t), min(t)+24])
    ax.set_xticks(range(24))
 
    ax.legend(loc='center right', ncol=1)

    configure_axes(ax)
        
    return plt

# Create the plot for all seasons and cases
fig = plot_power(res["test_base"])
save_plot(fig, "CHWpum_P")

### Outside conditions

In [86]:
def plot_outside(read_in_res):
    # ------------------------------------------------------
    plt.clf()
    plt.ylim([0., 20.])
    time_scale=86400.
    # ------------------------------------------------------
    
    ax = list()
    # get a list of unique climate cases
    unique_climate_cases = list(read_in_res.keys())[0] # all in the same climate
    
    subplot_id = 0
    for climate in unique_climate_cases:
        (t, TOut) = read_in_res[case_name].values("weaBus.TDryBul")
        (t, TOutWet) = read_in_res[case_name].values("weaBus.TWetBul")
        
        t = t/time_scale
        
        # Generate figure and plot data
        ax.insert(subplot_id, plt.subplot(len(unique_climate_cases)*2, 1, subplot_id+1))
        
        for subplot_id in [0,2,4]:
            ax[subplot_id].plot(t, TOut-273.15, 'r', \
                   label='$T_{oa}$', linewidth=0.5)
            ax[subplot_id].set_ylabel('temperature [$^\circ$C]')
            add_secondary_yaxis_for_degF(ax[subplot_id], t, TOut)
            set_title(ax[subplot_id], '$T_{oa}$')
        
        subplot_id += 1
        
        for subplot_id in [1,3,5]:
            ax[subplot_id].plot(t, TOutWet-273.15, 'r', \
                   label='$T_{oa,w}$', linewidth=0.5)
            ax[subplot_id].set_ylabel('temperature [$^\circ$C]')
            add_secondary_yaxis_for_degF(ax[subplot_id], t, TOutWet)
            set_title(ax[subplot_id], '$T_{oa,w}$')
        
        # customize days to display
        ax[subplot_id].set_xlim([min(t)+5, min(t)+10])

        configure_axes(ax[subplot_id])
        ax[subplot_id].legend(loc='center right', ncol=1)
        
        subplot_id += 1

#     ax[5].set_xlabel('time [days]')

    return plt

# Create the plot for all seasons and cases
fig = plot_outside(res)
save_plot(fig, "outside")

IndexError: list index out of range

### Energy use

We obtain the energy use as power consumed by:

- Zone supply fan: fan.P
- Chilled water pump: pumCHW.P
- Condenser water pump: pumCW.P
- Chiller: chi.P
- Cooling tower: cooTow.PFan



In [ ]:
from pdb import set_trace as bp

def plot_energy(res, cases):
    
    fan_P = dict()
    pumCHW_P = dict()
    pumCW_P = dict()
    chi_P = dict()
    cooTowFan_P = dict()
    for case in cases:
        (t, fan_P[case]) = res[case].values("fan.P")
        (t, pumCHW_P[case]) = res[case].values("pumCHW.P")
        (t, pumCW_P[case]) = res[case].values("pumCW.P")
        (t, chi_P[case]) = res[case].values("chi.P")
        (t, cooTowFan_P[case]) = res[case].values("cooTow.PFan")
        
    bp()

    plt.clf()    

    # Conversion to kWh/m2
    conv = 1/3600./1000.
    width = 0.5 # the width of the bars: can also be len(x) sequence

    zonSupFan = [0., 0.]
    pumCHW = [0., 0.]
    pumCW = [0., 0.]
    chiP = [0., 0.]
    cooTowFan = [0., 0.]
    
    idx = [0, 1]
    for i in idx:
        res_dic = res[i]
        zonSupFan[i] = res_dic['fan.P'][-1] * conv
        pumCHW[i] = res_dic['pumCHW.P'][-1] * conv
        pumCW[i] = res_dic['pumCW.P'][-1] * conv
        chiP[i] = res_dic['chi.P'][-1] * conv
        cooTowFan[i] = res_dic['cooTow.PFan'][-1] * conv
        
    p1 = plt.bar(idx, pumCHW + pumCW, width, color='r')
    p2 = plt.bar(idx, zonSupFan, width, bottom=hea, color='g')
    p3 = plt.bar(idx, chiP, width, bottom=cooLatBas, color='b')
    p4 = plt.bar(idx, cooTowFan, width, bottom=fanBas, color='k')
        
    plt.ylabel('site electricity use [kWh/(m2  a)]')
    plt.xticks([0, 1], ('test_base', 'test_1711'))
    plt.tick_params(axis=u'x', which=u'both',length=0)

    #plt.yticks(np.arange(0, 81, 10))
    plt.legend((p1[0], p2[0], p3[0], p4[0]), \
               ('pumping', 'chiller', 'cooling tower'), \
              loc='upper right')
    
    save_plot(plt, "energy")
    
    # Write result to console and file
    eSit = [0, 0]
    for i in [0, 1]:
        eSit[i] = hea[i]+cooSen[i]+cooLat[i]+fan[i]

#     str = """\
# .. table:: Heating, cooling, fan and total site HVAC energy, and savings of guideline 36 case versus base case.

#    ===================================== ===================================== ====================================== ====================================== =====
#    :math:`E_{{h}} \quad [kWh/(m^2\,a)]`    :math:`E_{{c}} \quad [kWh/(m^2\,a)]`    :math:`E_{{f}} \quad [kWh/(m^2\,a)]`     :math:`E_{{tot}} \quad [kWh/(m^2\,a)]`     [%]
#    ===================================== ===================================== ====================================== ====================================== =====
#    {:37.4} {:37.4}  {:37.4}  {:37.4}   
#    {:37.4} {:37.4}  {:37.4}  {:37.4}  {:4.3} 
#    ===================================== ===================================== ====================================== ====================================== =====
  
#   """.format(\
#             hea[0], coo[0], fan[0], eSit[0], \
#             hea[1], coo[1], fan[1], eSit[1], (1-eSit[1]/eSit[0])*100.)
#     def save_rst(str, file_name):
#         ''' Save the string `str` to the rst file `file_name.rst`
#         '''
#         print(str)
#         with open(os.path.join("img", "{}.rst".format(file_name)), "w") as fil:
#             fil.write(str)
        
#     save_rst(str, "site_energy")
cases = [0, 1]
plot_energy(res, cases)

> <ipython-input-28-560a2bb4ec5f>(19)plot_energy()
     17     bp()
     18 
---> 19     plt.clf()
     20 
     21     # Conversion to kWh/m2



ipdb>  cooTowFan_P


{0: array([6000., 6000.], dtype=float32), 1: array([6000., 6000.], dtype=float32)}


ipdb>  chi_P


{0: array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       3.1341208e+03, 1.8279717e+03, 2.8036202e-05, 2.8294497e-05,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000

## Plot energy use with diverse loads

In [ ]:
def plot_all_energy(r_base_without, r_g36_without, r_base_with, r_g36_with):
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt
    
    plt.clf()    

    # Conversion to kWh/m2
    conv = 1/3600./1000.
    results = [r_base_without, r_g36_without, r_base_with, r_g36_with]
    width = 0.5       # the width of the bars: can also be len(x) sequence

    hea    = [0., 0., 0., 0.]
    cooSen = [0., 0., 0., 0.]
    cooLat = [0., 0., 0., 0.]
    fan    = [0., 0., 0., 0.]
    cooLatBas = [0., 0., 0., 0.]
    coo = [0, 0, 0., 0.]
    fanBas = [0., 0., 0., 0.]
    
    idx = [0, 1, 2, 3]
    for i in idx:
        res_dic = results[i]
        hea[i]    =  res_dic['res.EHea'][-1] * conv / COPh
        cooSen[i] = -res_dic['res.ECooSen'][-1] * conv / COPc
        cooLat[i] = -res_dic['res.ECooLat'][-1] * conv / COPc
        coo[i] = cooSen[i] + cooLat[i]
        fan[i]    =  res_dic['res.EFan'][-1] * conv
        cooLatBas[i] = hea[i] + cooSen[i]
        fanBas[i]    = cooLatBas[i] + cooLat[i]
    
    
    plt.figure(figsize=(5,3))

    p1 = plt.bar(idx, hea,    width, color='r')
    p2 = plt.bar(idx, cooSen, width, bottom=hea, color='g')
    p3 = plt.bar(idx, cooLat, width, bottom=cooLatBas, color='b')
    p4 = plt.bar(idx, fan,    width, bottom=fanBas, color='k')
    plt.ylabel('site electricity use [kWh/(m2  a)]')
    plt.xticks([0, 1, 2, 3], ('base case', 'guideline 36', 'base case, $\pm 50\%$', 'guideline 36, $\pm 50\%$'))
    #plt.yticks(np.arange(0, 81, 10))
    plt.legend((p1[0], p2[0], p3[0], p4[0]), \
               ('heating', 'sensible cooling', 'latent cooling', 'fan'), \
              loc='upper right')
    plt.grid(color='lightgrey', axis='y', linewidth=0.25)
    for spine in plt.gca().spines.values():
        spine.set_visible(False)
    save_plot(plt, "energy_all")
    
plot_all_energy(r_base_annual_without_diverse_loads, r_g36_annual_without_diverse_loads, \
                r_base_annual_with_diverse_loads,    r_g36_annual_with_diverse_loads)

### Diagnostic output

In [ ]:
print("CPUtime, base {} h".format(r_base_annual_without_diverse_loads['CPUtime'][-1]/3600.))
print("CPUtime, G36  {} h".format(r_g36_annual_without_diverse_loads['CPUtime'][-1]/3600.))

In [ ]:
plt.clf()

roo_lis = [{"var": 'nor', "name": 'north'},
           {"var": 'wes', "name": 'west'},
           {"var": 'sou', "name": 'south'},
           {"var": 'eas', "name": 'east'},
           {"var": 'cor', "name": 'corridor'}]
nRoo = len(roo_lis)
iRoo = 1
iPlt = 1
for roo in roo_lis:
    iSim = 1
    for sim in [{"res": r_base_annual_without_diverse_loads, 'name': 'base case'}, 
                {"res": r_g36_annual_without_diverse_loads, 'name': 'guideline36'}]:
        plt.subplot(nRoo, 2, iPlt)
        plt=Plotter.boxplot(t=sim['res']['time'], \
                    y=sim['res']['flo.{}.air.vol.T'.format(roo['var'])]-273.15, \
                    increment=3600, nIncrement=24)
        #plt.set_facecolor('mistyrose')    
        #plt.patch.fill_between(t, TSetHea-273.15, y2=TSetCoo-273.15, color='white')
        # Decorate, save and show the plot
        if iRoo is nRoo:
            plt.xlabel('Time [h]') 
        if iSim is 1:
            plt.ylabel(u'$T_{room}$ [$^\circ$C]')
        plt.grid()
        plt.xlim(6.5,19.5)
        plt.ylim(18, 26)
        plt.yticks([18, 20, 22, 24, 26])

        plt.title('{} zone, {}'.format(roo['name'], sim['name']))
        iSim+=1
        iPlt+=1
    iRoo+=1
plt.tight_layout()
save_plot(plt, "roomTemperatures_boxplot")
plt.show()

In [7]:
a = cases.get_cases()

In [6]:
a

<function cases.get_cases()>

['test_base', 'test_1711']